In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import pandas
import os

%matplotlib inline
data_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/test'
filenames = [os.path.join(data_dir, 'data_test_batch.bin')]
filename_queue = tf.train.string_input_producer(filenames)
batch_size =100
# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 5
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 2000 # was set from # 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 3500
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

Using TensorFlow backend.


In [7]:

class SVHNRecord(object):
    pass

result = SVHNRecord()
result.height = 224
result.width = 224
result.depth = 3
num_exmaples = 1
reader = tf.TFRecordReader()
result.key, value = reader.read(filename_queue)
value = tf.parse_single_example(
    value,
    # Defaults are not specified since both keys are required.
    features={
        'image_raw': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'label': tf.FixedLenFeature(shape=[], dtype=tf.int64),
    })

# Convert from a string to a vector of uint8 that is record_bytes long.
record_bytes = tf.decode_raw(value['image_raw'], tf.uint8)
# record_bytes.set_shape([32*32*3])
record_bytes = tf.reshape(record_bytes, [1,256*256*3])
print("record bytes::::: ", record_bytes)
# Store our label to result.label and convert to int32
result.label = tf.cast(value['label'], tf.int32)
result.label = tf.reshape(result.label,[num_exmaples])
result.uint8image = record_bytes

('record bytes::::: ', <tf.Tensor 'Reshape_1:0' shape=(1, 196608) dtype=uint8>)


In [8]:
result.label

<tf.Tensor 'Reshape_2:0' shape=(1,) dtype=int32>

In [ ]:
# We can now access the default graph where all our metadata has been loaded

In [ ]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    print(sess.run(result.uint8image).shape)
    print(sess.run(result.label))


In [9]:
# Ensure that the random shuffling has good mixing properties.
min_fraction_of_examples_in_queue = 0.4
min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN *
                         min_fraction_of_examples_in_queue)

In [10]:
images, label_batch = tf.train.batch(
    [result.uint8image, result.label],
    batch_size=batch_size,
    num_threads=16,
    capacity=min_queue_examples + 3 * batch_size)

In [ ]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    print(sess.run(images).shape)
    #print(sess.run(result.label))

In [11]:

target_probs = np.array([77,  5, 12,  4,  2], dtype=np.float32)/100
target_probs


array([0.77, 0.05, 0.12, 0.04, 0.02], dtype=float32)

In [ ]:
mask = np.array([True, True, False])

In [ ]:
labelt = tf.boolean_mask(
    label_batch,
    mask=mask,
    name='boolean_mask',
    axis=None
)

In [ ]:
sort = tf.nn.top_k(
    label_batch,
    k=1,
    sorted=True,
    name=None
)

In [ ]:
shuffled_data, shuffled_labels = tf.train.slice_input_producer(
  [images, label_batch])

In [ ]:
shuffled_labels
target_probs.reshape(5,1).shape

In [ ]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    labels = sess.run(label_batch)
    print(np.unique(labels, return_counts=True))
    print(labels)


In [13]:
dataset = tf.data.Dataset.from_tensor_slices((images, label_batch))
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
        
    iterator = dataset.make_initializable_iterator()
    next_element = iterator.get_next()
    
    # Initialize an iterator over a dataset with 10 elements.
    sess.run(iterator.initializer)
    for i in range(10):
        value = sess.run(next_element)
        print(value[1])

[2]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


In [14]:
tf.data.Dataset.from_tensors(dataset)

TypeError: Failed to convert object of type <class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'> to Tensor. Contents: <TensorSliceDataset shapes: ((1, 196608), (1,)), types: (tf.uint8, tf.int32)>. Consider casting elements to a supported type.

In [12]:
[x,y] = tf.contrib.training.stratified_sample([images],
labels=label_batch,
target_probs=target_probs,
batch_size=30,
init_probs=None,
enqueue_many=False,
queue_capacity=16,
threads_per_queue=1,
name=None)

ValueError: Shapes must be equal rank, but are 1 and 3 for 'stratified_sample/AssignAdd' (op: 'AssignAdd') with input shapes: [5], [100,1,5].

In [ ]:
x


In [ ]:
images_sl = tf.train.slice_input_producer(
  [images], shuffle=True, capacity=3*batch_size)
print(images_sl)

In [ ]:
import numpy
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

with tf.Graph().as_default():
    batch_size = 100
    data = ['a']*9000+['b']*1000
    labels = [1]*9000+[0]*1000
    data_tensor = ops.convert_to_tensor(data, dtype=dtypes.string)
    label_tensor = ops.convert_to_tensor(labels, dtype=dtypes.int32)
    print(data_tensor)
    shuffled_data, shuffled_labels = tf.train.slice_input_producer(
      [data_tensor, label_tensor], shuffle=True, capacity=3*batch_size)
    print(shuffled_data)
    target_probs = numpy.array([0.5,0.5])
    data_batch, label_batch = tf.contrib.training.stratified_sample(
      [shuffled_data], shuffled_labels, target_probs, batch_size,
      queue_capacity=2*batch_size)

    with tf.Session() as session:
        tf.local_variables_initializer().run()
        tf.global_variables_initializer().run()
        coordinator = tf.train.Coordinator()
        tf.train.start_queue_runners(session, coord=coordinator)
        num_iter = 10
        sum_ones = 0.
        for _ in range(num_iter):
            d, l = session.run([data_batch, label_batch])
            count_ones = l.sum()
            sum_ones += float(count_ones)
            print('percentage "a" = %.3f' % (float(count_ones) / len(l)))
        print('Overall: {}'.format(sum_ones / (num_iter * batch_size)))
        coordinator.request_stop()
        coordinator.join()

In [ ]:
images